In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pandas as pd
import re
from itertools import chain
import joblib

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joao.victor.ribeiro\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joao.victor.ribeiro\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
stop_words = set(stopwords.words('english'))
wnet = nltk.WordNetLemmatizer()

In [2]:
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

In [3]:
genres = [
    'Country',
    'Rap',
    'Rock'
]

In [4]:
loaded_models = {}
for genre in genres:
    filename = f'{genre}_model.joblib'
    loaded_models[genre] = joblib.load(filename)

In [11]:
vectorize = joblib.load('tfidf_vectorizer.joblib')

In [26]:
file_path = 'lyric.txt'
text_content = read_txt_file(file_path)
df = pd.DataFrame({'lyrics': [text_content]})
df

,lyrics
0,I used to spend my nights out in a barroom\nLi...


In [27]:
text_content = read_txt_file(file_path)
df = pd.DataFrame({'lyrics': [text_content]})
tokenized = [word_tokenize(lyr) for lyr in df['lyrics'].astype(str)]
stop_vec = [[w for w in tok if w not in stop_words] for tok in tokenized]
clean_vec = [[word for word in lyr if word.isalpha()] for lyr in stop_vec]
lem = [[wnet.lemmatize(w) for w in lyr] for lyr in clean_vec]

In [28]:
lyrics_tay = [' '.join(lyr) for lyr in lem]
single_entry = vectorize.transform(lyrics_tay)
s_e = single_entry.todense().tolist()

probabilities = {genre: loaded_models[genre].predict_proba(s_e)[0][1] for genre in genres}

c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


In [29]:
for genre, prob in probabilities.items():
    print(f"{genre}: {prob * 100:.2f}%")

Country: 80.87%
Rap: 10.04%
Rock: 47.82%
